In [1]:
import torch
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model, assemble_model_with_classifier
from model.utils import freeze_layers
from trainer.trainer import get_trainer

from model.resnet import *

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# redirect print to file
# import sys
# sys.stdout = open("PyTorch-resnet34-log.txt", "w")

In [2]:
# configuration
config = {
    'train_batch_size': 512, 'val_batch_size': 512,
    'arch': 'resnet152', 'pretrained': True,
    'optimizer': 'Adam', 'learning_rate': 1e-3, 'decay_lr_freq': 2e4, 'weight_decay': 1e-5,
    'resume': None,
    'start_epoch': 0, 'epochs': 10,
    'print_freq': 10, 'validate_freq': 2e4, 'save_freq': 1e3,
    'best_val_prec1': 0
}

In [3]:
import torchvision.models as models

# get dataset
print('getting dataset...')
train_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                      images_csv="train_images.csv",
                                      bson_file_path="/mnt/data/cdiscount/train.bson",
                                      with_label=True,
                                      resize=160)
val_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                    images_csv="val_images.csv",
                                    bson_file_path="/mnt/data/cdiscount/train.bson",
                                    with_label=True,
                                    resize=160)

# get data loader
print('getting data loader...')
train_dataloader = DataLoader(train_dataset, batch_size=config['train_batch_size'], shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=config['val_batch_size'], shuffle=True, num_workers=1)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
pretrained_model = models.__dict__[config['arch']](pretrained=config['pretrained'])

# model = torch.nn.Sequential(*list(model.children())[:-1])
# model.add_module('classifier', torch.nn.Linear(in_features=2048, out_features=5270))
# model = torch.nn.DataParallel(model).cuda()
# print(pretrained_model)

model = resnet152(num_classes=5270)

pretrained_state = pretrained_model.state_dict()
model_state = model.state_dict()

for name, state in pretrained_state.items():
    if not name.startswith('fc'):
        model_state[name].copy_(state)

freeze_layers(model, 7)

model = torch.nn.DataParallel(model).cuda()
print(model)

getting dataset...
getting data loader...
=> using pre-trained model 'resnet152'
0 freezing Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
1 freezing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
2 freezing ReLU (inplace)
3 freezing MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
4 freezing Sequential (
  (0): Bottleneck (
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (downsample): Sequential (
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, mome

DataParallel (
  (module): ResNet (
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (layer1): Sequential (
      (0): Bottleneck (
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU (inplace)
        (downsample): Sequential (
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e

In [ ]:
# define loss function (criterion) and optimizer
criterion = torch.nn.CrossEntropyLoss().cuda()

# get trainer
Trainer = get_trainer(train_dataloader, val_dataloader, model, criterion, config)

# Run!
Trainer.run()

start training
Epoch: [0][0/19340]	Time 34.280 (34.280)	Data 2.510 (2.510)	Loss 8.6566 (8.6566)	Prec@1 0.000 (0.000)	Prec@5 0.195 (0.195)
Epoch: [0][10/19340]	Time 4.896 (7.539)	Data 0.019 (0.247)	Loss 7.2203 (8.1661)	Prec@1 6.250 (4.688)	Prec@5 14.258 (10.973)
Epoch: [0][20/19340]	Time 4.919 (6.288)	Data 0.020 (0.137)	Loss 6.4637 (7.5279)	Prec@1 11.133 (6.538)	Prec@5 21.680 (14.788)
Epoch: [0][30/19340]	Time 4.930 (5.847)	Data 0.014 (0.097)	Loss 6.2831 (7.1204)	Prec@1 12.500 (8.380)	Prec@5 25.977 (18.095)
Epoch: [0][40/19340]	Time 4.941 (5.626)	Data 0.014 (0.077)	Loss 5.6753 (6.8180)	Prec@1 16.602 (9.775)	Prec@5 29.688 (20.736)
